<a href="https://colab.research.google.com/github/ronwuyk/LSTM-learning/blob/master/capstonev3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To install TensorFlow, you can use pip, which is a package installer for Python.

# Install the CPU version of TensorFlow
!pip install tensorflow

# If you have a compatible NVIDIA GPU and want to install the GPU version of TensorFlow, use the following command
!pip install tensorflow-gpu

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load dataset
df = pd.read_csv('/content/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

# Preprocess data
df['lease_commence_date'] = pd.to_datetime(df['lease_commence_date'], format='%Y')
df['remaining_lease'] = df['lease_commence_date'].dt.year + 99 - pd.to_datetime(df['month'], format='%Y-%m').dt.year

# Select relevant columns
features = ['town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'remaining_lease']
X = df[features]
y = df['resale_price']

# Encode categorical features
X = pd.get_dummies(X, columns=['town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {loss}')

# Function to predict resale price based on user input
def predict_resale_price(user_input):
    user_df = pd.DataFrame([user_input], columns=features)
    user_df = pd.get_dummies(user_df)
    user_df = user_df.reindex(columns=X.columns, fill_value=0)
    user_df = scaler.transform(user_df)
    prediction = model.predict(user_df)
    return prediction[0][0]

# Example user input
user_input = {
    'town': 'ANG MO KIO',
    'flat_type': '3 ROOM',
    'block': '123',
    'street_name': 'ANG MO KIO AVE 3',
    'storey_range': '06 TO 10',
    'floor_area_sqm': 67,
    'flat_model': 'Improved',
    'remaining_lease': 60
}

predicted_price = predict_resale_price(user_input)
print(f'Predicted Resale Price: {predicted_price}')


Epoch 1/100
3703/3703 [==============================] - 16s 4ms/step - loss: 314011.6875 - val_loss: 216122.2031
Epoch 2/100
3703/3703 [==============================] - 17s 5ms/step - loss: 185644.0312 - val_loss: 171720.0156
Epoch 3/100
3703/3703 [==============================] - 17s 5ms/step - loss: 159085.2656 - val_loss: 156735.3438
Epoch 4/100
3703/3703 [==============================] - 19s 5ms/step - loss: 147085.1875 - val_loss: 147620.6562
Epoch 5/100
3703/3703 [==============================] - 14s 4ms/step - loss: 138612.1094 - val_loss: 140227.0156
Epoch 6/100
3703/3703 [==============================] - 14s 4ms/step - loss: 130945.7031 - val_loss: 132618.1562
Epoch 7/100
3703/3703 [==============================] - 14s 4ms/step - loss: 122843.7969 - val_loss: 124263.9375
Epoch 8/100
3703/3703 [==============================] - 14s 4ms/step - loss: 114138.1172 - val_loss: 115574.1172
Epoch 9/100
3703/3703 [==============================] - 15s 4ms/step - loss: 105567.375

In [5]:
!pip install ipywidgets

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import ipywidgets as widgets
from IPython.display import display

# ... (rest of your data loading and model training code)

# Function to predict resale price based on user input (unchanged)
def predict_resale_price(user_input):
    user_df = pd.DataFrame([user_input], columns=features)
    user_df = pd.get_dummies(user_df)
    user_df = user_df.reindex(columns=X.columns, fill_value=0)
    user_df = scaler.transform(user_df)
    prediction = model.predict(user_df)
    return prediction[0][0]

# GUI Implementation with ipywidgets
towns = df['town'].unique().tolist()
flat_types = df['flat_type'].unique().tolist()
# ... (get unique values for other features)

town_dropdown = widgets.Dropdown(options=towns, description='Town:')
flat_type_dropdown = widgets.Dropdown(options=flat_types, description='Flat Type:')
# ... (create dropdown widgets for other features)

area_text = widgets.FloatText(value=67, description='Floor Area (sqm):')
lease_text = widgets.IntText(value=60, description='Remaining Lease:')

result_label = widgets.Label()

def on_predict(button):
    user_input = {
        'town': town_dropdown.value,
        'flat_type': flat_type_dropdown.value,
        # ... (get values from other widgets)
        'floor_area_sqm': area_text.value,
        'remaining_lease': lease_text.value
    }
    predicted_price = predict_resale_price(user_input)
    result_label.value = f'Predicted Resale Price: {predicted_price:.2f}'

predict_button = widgets.Button(description="Predict")
predict_button.on_click(on_predict)

display(town_dropdown, flat_type_dropdown,  # ... (display other widgets)
        area_text, lease_text, predict_button, result_label)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00


Dropdown(description='Town:', options=('ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH', 'BUKIT P…

Dropdown(description='Flat Type:', options=('2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM', 'MU…

FloatText(value=67.0, description='Floor Area (sqm):')

IntText(value=60, description='Remaining Lease:')

Button(description='Predict', style=ButtonStyle())

Label(value='')

1/1 [==============================] - 0s 23ms/step


GUI for User to interact with the programme